In [222]:

import pandas as pd
import numpy as np
import time
import seaborn as sb
import datetime as dt
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta
from dateutil import tz

In [9]:
def format_date(t):
    if len(t)<11:
        t = t+' 12:00' #somehow the data is missing the 12pm timem stamp
    date,time = t.split(' ')
    month,day,year = date.split('/')
    hr,sec = time.split(':')

    return dt.datetime(int(year),int(month),int(day),int(hr),int(sec))
        
     

In [247]:

def is_local_max(data, index):
    if(data[index-1][4]<=data[index][4] and data[index][4]>=data[index+1][4]):# and data[index-1][1]<=data[index][4]:

        return True
    return False
def is_local_min(data, index):
    if(data[index-1][4]>=data[index][4] and data[index][4]<=data[index+1][4]):
        return True
    return False
def calc_gradient(pt1,pt2):
    return (pt2[1]-pt1[1])/(pt2[0]-pt1[0])

def detect_asc_triangle2(data, max_size, max_tolerance, number_of_max, number_of_low,min_grad,look_back =15,max_grad = 0.05/290*5 ):
    #max size indicates number of candle sticks
    #max tolerance in BP
    tolerance = max_tolerance/10000
    max_list = []
    min_list = []
    #i want to detect a serious of max
    for i in range(1,len(data)-1):

        if is_local_max(data,i):
            max_list.append((data[i],i))
        
        if is_local_min(data,i):
            min_list.append((data[i],i))

    triangle_max = []
    index = 0
    while index < len(max_list):

        triangle_min = []
        triangle_max = [max_list[index]]
        not_triangle = False
        j = 1
        last_j = 1
        break_out = 0

        while j+index<=len(max_list)-1 and max_list[index+j][1]<=max_list[index][1]+max_size: 

            if max_list[index+j][0][4]<=max_list[index][0][4]*(1+tolerance) and max_list[index+j][0][4]>=max_list[index][0][4]*(1-tolerance):
                break_out = max_list[index+j][1]
                triangle_max.append(max_list[index+j])
                last_j = j


            if max_list[index+j][0][4]>max_list[index][0][4]*(1+tolerance):

                break_out = max_list[index+j][1]

                break

            j+=1


        if len(triangle_max) < number_of_max: 

            index = index +1
            continue

        if (triangle_max[-1][1] -triangle_max[0][1])<10: # triangel likely to be too small
            index = index +1
            continue
        for i in range(len(min_list)):
            if min_list[i][1]< triangle_max[0][1]-look_back: #too far back
                continue
            if (min_list[i][1]>break_out) :
                break
            triangle_min.append(min_list[i])

        if len(triangle_min)<2:
            index= index+1
  
            continue

        low_index = 0
        for count in range(1,len(triangle_min)): # find the lowest value traingle 
            if triangle_min[count][0][4]<triangle_min[low_index][0][4]:
                low_index = count

        low_grad = 9999
        second_index  = 0 
        if low_index == len(triangle_min)-1:
            index = index+1
            print("lowest < triangle min")
            continue
        
        for count in range(low_index+1, len(triangle_min)):
            grad = calc_gradient((triangle_min[low_index][1],triangle_min[low_index][0][4]),\
                                (triangle_min[count][1],triangle_min[count][0][4]))


            if grad<low_grad:
                low_grad = grad
                second_index = count

        if low_grad>max_grad*triangle_min[low_index][0][4]:#use grad* price

            index = index+last_j
  
            continue
        if low_grad<min_grad:
            index = index+1
            continue
            
        if triangle_min[low_index][1] - triangle_max[0][1]>30: # min too far away from max
            index = index + 1
            continue


        print("triangle with max points{} and low points @ {} and @{}"\
              .format(str(triangle_max),triangle_min[low_index],triangle_min[second_index]))


        index = index+last_j


In [249]:
#sanity check should be  a 3 no trades while last one should hav one winning trade
def convert_iso_NY_datetime(date):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')
    utc = dt.datetime.strptime(date,"%Y-%m-%dT%H:%M:%SZ")
    utc = utc.replace(tzinfo=from_zone)
    central = utc.astimezone(to_zone)
    return central


data = pd.read_csv('spy5min.csv')



asc_data = data.to_numpy()
detect_asc_triangle2(asc_data, 40, 10,2,1,0.03)
print("hello")

triangle with max points[(array(['2020-03-10T15:05:00Z', 278.94, 280.31, 278.75, 280.08, 263960,
       307706.45, nan, nan, nan, nan], dtype=object), 19), (array(['2020-03-10T17:20:00Z', 279.93, 280.89, 279.85, 280.09, 531984,
       286723.1, nan, nan, nan, nan], dtype=object), 46), (array(['2020-03-10T17:30:00Z', 279.27, 280.25, 278.53, 280.03, 382165,
       291666.0, nan, nan, nan, nan], dtype=object), 48)] and low points @ (array(['2020-03-10T15:30:00Z', 274.92, 275.45, 274.27, 274.58, 563588,
       307571.15, nan, nan, nan, nan], dtype=object), 24) and @(array(['2020-03-10T16:30:00Z', 277.55, 277.66, 275.79, 276.29, 373465,
       323724.65, nan, nan, nan, nan], dtype=object), 36)
lowest < triangle min
lowest < triangle min
triangle with max points[(array(['2020-03-19T15:45:00Z', 244.96, 245.58, 244.25, 245.46, 392750,
       370387.15, nan, nan, nan, nan], dtype=object), 567), (array(['2020-03-19T17:50:00Z', 243.27, 245.81, 243.25, 245.56, 389611,
       266955.2, nan, nan, na